In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [ ]:
p1=pd.read_csv('https://raw.githubusercontent.com/gelya1709/customer_segmentation/main/20210521_offers.csv')

In [ ]:
p1['start_date']=p1['start_date'].astype(str)
p1['start_date'] = p1['start_date'].apply(lambda x: dt.datetime.strptime(x,'%Y%m%d'))
p1['end_date']=p1['end_date'].astype(str)
p1['end_date'] = p1['end_date'].apply(lambda x: dt.datetime.strptime(x,'%Y%m%d'))
p1.head()

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,2020-01-02,2020-01-08,train
2,Facades,F9,5ae92b5e465d630285677e80b43fe5f7,2020-01-16,2020-01-22,train
3,Facades,F45,0790b472112a765339f0e5c3f66816ee,2020-03-19,2020-03-25,train
4,Facades,F191,e87788b585ead792a4d40f7de18f2f76,2020-11-12,2020-11-18,test


In [ ]:
p1['start_date'] = pd.to_datetime(p1['start_date'])
p1['end_date'] = pd.to_datetime(p1['end_date'])

In [ ]:
p1['start_date'].min() 

Timestamp('2019-09-24 00:00:00')

In [ ]:
p1=p1[(p1['start_date']>=dt.datetime(2019,9,20)) & (p1['start_date']<dt.datetime(2020,9,20))]

In [ ]:
year_copy_19=p1[p1['start_date']<=dt.datetime(2019,12,29)]
year_copy_20=p1[p1['start_date']>dt.datetime(2019,12,29)]
year_copy_19['week_1']=[i.isocalendar()[1] for i in year_copy_19['start_date']]
year_copy_20['week_1']=[i.isocalendar()[1]+52 for i in year_copy_20['start_date']]
p1 = pd.concat([year_copy_19, year_copy_20], ignore_index=True)

In [ ]:
year_copy_19=p1[p1['end_date']<=dt.datetime(2019,12,29)]
year_copy_20=p1[p1['end_date']>dt.datetime(2019,12,29)]
year_copy_19['week_2']=[i.isocalendar()[1] for i in year_copy_19['end_date']]
year_copy_20['week_2']=[i.isocalendar()[1]+52 for i in year_copy_20['end_date']]
num_promo = pd.concat([year_copy_19, year_copy_20], ignore_index=True)

In [ ]:
num_promo['combined'] = num_promo.apply(lambda x: list([x['week_1'], x['week_2']]),axis=1) 
num_promo.head()

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,week_1,week_2,combined
0,Billboards,B116,802613d2fbd18f89fb84acc0ae0c14e5,2019-12-12,2019-12-25,train,50,52,"[50, 52]"
1,Billboards,B115,119a9ea3a832472e1c163e5c1afba59e,2019-12-12,2019-12-25,train,50,52,"[50, 52]"
2,Billboards,B116,330237fcfba03e0772a1c1d00c0a7943,2019-12-12,2019-12-25,train,50,52,"[50, 52]"
3,Billboards,B110,9817112604594b376adf135efe787bbc,2019-11-28,2019-12-11,train,48,50,"[48, 50]"
4,Billboards,B112,753e100607ecf3e68098739b81e0a645,2019-11-28,2019-12-04,train,48,49,"[48, 49]"


In [ ]:
weeks=[]
for i in num_promo['combined']:
  if i[0]==i[1]:
    weeks.append(i[0])
  else:
    weeks.append(list(range(i[0], i[1]+1)))
num_promo['all_weeks']=weeks

In [ ]:
num_promo.head()

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,week_1,week_2,combined,all_weeks
0,Billboards,B116,802613d2fbd18f89fb84acc0ae0c14e5,2019-12-12,2019-12-25,train,50,52,"[50, 52]","[50, 51, 52]"
1,Billboards,B115,119a9ea3a832472e1c163e5c1afba59e,2019-12-12,2019-12-25,train,50,52,"[50, 52]","[50, 51, 52]"
2,Billboards,B116,330237fcfba03e0772a1c1d00c0a7943,2019-12-12,2019-12-25,train,50,52,"[50, 52]","[50, 51, 52]"
3,Billboards,B110,9817112604594b376adf135efe787bbc,2019-11-28,2019-12-11,train,48,50,"[48, 50]","[48, 49, 50]"
4,Billboards,B112,753e100607ecf3e68098739b81e0a645,2019-11-28,2019-12-04,train,48,49,"[48, 49]","[48, 49]"


In [ ]:
num_promo['Promo_type'].unique()

array(['Billboards', 'Facades', 'Seasonal', 'Biweekly'], dtype=object)

In [ ]:
a=num_promo[num_promo['Promo_type']=='Biweekly']

slovar={}
listt=[i for i in range(39,91)]
for i in listt:
  num=0
  
  for item in a['all_weeks']:
    if type(item)==int:
      if item==i:
        num+=1
    if type(item)==list:
      if i in item:
        num+=1
    slovar[i]=num
#stat_promo=pd.DataFrame()
#stat_promo['week']=slovar.keys()
stat_promo['count_promo_Biweekly']=slovar.values()

In [ ]:
stat_promo.shape

(52, 6)

In [ ]:
num_promo.group_by('week')

In [ ]:
stat_promo=stat_promo.iloc[1:,:]

In [ ]:
sleeping_to_sleeping = [39.5, 32.9, 33.2, 32.5, 33.4, 32.6, 32.5, 33.0, 32.7, 34.3, 34.5, 34.2, 33.5, 39.1, 29.2, 30.4, 33.0, 35.0, 34.1, 37.0, 35.2, 37.5, 33.1, 35.3, 29.9, 36.8, 34.3, 30.1, 36.0, 37.7, 33.7, 37.0, 38.5, 36.2, 37.3, 31.6, 37.1, 36.7, 36.0, 35.4, 35.5, 35.4, 35.2, 34.7, 33.9, 34.8, 36.4, 35.2, 31.0, 34.0, 32.1, 32.3]
sleeping_to_loyal = [5.0, 7.7, 5.8, 6.0, 6.4, 6.2, 6.7, 6.5, 6.7, 6.7, 6.1, 6.8, 6.7, 7.3, 6.3, 5.6, 7.0, 6.1, 6.3, 4.8, 8.4, 3.1, 7.5, 8.6, 5.2, 5.9, 6.5, 5.1, 8.6, 7.3, 7.4, 6.5, 5.9, 5.0, 6.0, 12.4, 4.4, 5.9, 5.4, 5.0, 5.9, 5.4, 5.5, 5.1, 5.1, 5.0, 4.7, 4.9, 7.7, 4.9, 6.9, 5.8]
sleeping_to_champions=[5.0, 3.6, 4.6, 4.1, 4.1, 4.4, 4.2, 4.0, 4.5, 4.6, 4.0, 5.4, 5.0, 3.6, 3.8, 4.0, 4.7, 3.9, 4.8, 2.5, 4.6, 5.4, 4.6, 5.7, 4.5, 2.9, 3.1, 2.7, 5.1, 2.9, 5.2, 3.0, 3.9, 5.1, 3.0, 2.8, 3.5, 3.2, 3.6, 3.4, 3.5, 3.1, 3.5, 3.0, 2.9, 3.4, 3.4, 3.0, 4.8, 2.6, 4.4, 4.7]
sleeping_to_churn=[50.5, 55.7, 56.3, 57.5, 56.1, 56.8, 56.5, 56.5, 56.1, 54.4, 55.4, 53.6, 54.8, 50.0, 60.7, 60.0, 55.3, 55.1, 54.8, 55.6, 51.8, 54.0, 54.7, 50.4, 60.4, 54.5, 56.2, 62.0, 50.3, 52.2, 53.6, 53.4, 51.7, 53.7, 53.7, 53.3, 55.0, 54.3, 55.0, 56.1, 55.2, 56.1, 55.8, 57.2, 58.1, 56.8, 55.5, 56.9, 56.5, 58.5, 56.5, 57.2]

loyal_to_sleeping=[40.3, 27.0, 32.8, 33.1, 33.0, 32.3, 33.1, 35.2, 33.2, 33.3, 35.6, 32.6, 33.4, 42.2, 28.9, 35.5, 32.1, 35.2, 32.9, 45.9, 27.4, 45.5, 26.9, 31.5, 35.3, 43.5, 27.0, 25.3, 28.2, 43.2, 26.5, 42.7, 27.8, 26.3, 25.7, 17.9, 34.5, 27.9, 26.4, 29.3, 28.9, 27.5, 27.9, 26.6, 26.0, 27.0, 25.5, 25.6, 24.4, 43.3, 24.2, 32.4]
loyal_to_loyal=[6.2, 5.9, 38.4, 38.2, 39.9, 37.5, 35.8, 37.1, 38.1, 39.2, 36.7, 37.7, 38.1, 6.9, 5.2, 29.3, 39.0, 37.5, 38.6, 5.1, 6.2, 26.7, 50.4, 41.8, 30.8, 6.0, 22.1, 19.9, 6.0, 8.7, 5.9, 6.9, 22.5, 22.3, 22.5, 20.3, 12.2, 22.7, 21.0, 21.4, 19.9, 20.2, 20.7, 19.7, 19.4, 19.4, 18.7, 19.7, 5.2, 5.8, 4.5, 34.5]
loyal_to_champions=[19.5, 17.1, 3.7, 5.4, 5.0, 5.1, 5.5, 4.2, 4.6, 5.5, 4.5, 6.0, 4.9, 28.0, 9.8, 5.1, 4.8, 4.4, 5.3, 26.6, 19.4, 6.5, 5.8, 6.7, 5.2, 27.7, 2.9, 2.6, 19.4, 26.9, 21.0, 26.8, 2.8, 4.1, 3.1, 14.2, 9.9, 2.9, 2.3, 3.4, 3.2, 2.8, 3.0, 2.4, 2.2, 3.1, 3.0, 1.9, 19.6, 25.3, 19.7, 5.3]
loyal_to_churn=[34.0, 50.0, 25.0, 23.3, 22.1, 25.2, 25.7, 23.6, 24.1, 22.0, 23.2, 23.7, 23.5, 22.9, 56.1, 30.0, 24.1, 22.9, 23.2, 22.4, 47.0, 21.3, 16.9, 20.0, 28.7, 22.7, 48.0, 52.1, 46.4, 21.3, 46.6, 23.6, 46.9, 47.2, 48.7, 47.6, 43.4, 46.5, 50.3, 45.9, 48.1, 49.5, 48.4, 51.3, 52.4, 50.6, 52.8, 52.8, 50.8, 25.6, 51.6, 27.8]

champions_to_sleeping= [26.0, 28.2, 23.4, 24.4, 26.9, 24.9, 28.6, 26.0, 26.3, 25.5, 27.3, 26.1, 28.1, 29.8, 32.0, 28.4, 24.1, 27.1, 24.7, 27.1, 26.1, 29.6, 26.0, 26.4, 27.0, 29.0, 38.3, 37.3, 28.5, 23.7, 26.1, 25.8, 36.7, 35.6, 43.3, 34.4, 25.0, 39.0, 37.9, 38.8, 40.9, 35.9, 36.8, 35.7, 37.0, 38.6, 39.8, 37.1, 25.5, 26.5, 28.4, 25.9]
champions_to_loyal=[18.8, 47.4, 6.5, 4.9, 6.0, 6.0, 5.3, 6.1, 5.4, 5.7, 5.9, 5.8, 6.1, 20.8, 34.4, 6.0, 5.5, 5.3, 4.8, 21.1, 55.1, 3.2, 6.3, 8.1, 5.5, 21.3, 7.0, 6.3, 46.9, 25.0, 50.4, 23.9, 8.2, 7.9, 6.3, 41.7, 26.5, 5.6, 7.1, 6.0, 5.8, 5.8, 6.0, 5.8, 6.0, 5.2, 5.8, 6.0, 52.6, 20.2, 45.6, 4.8]
champions_to_champions=[3.6, 5.3, 20.6, 16.9, 18.3, 18.9, 18.3, 19.3, 19.9, 21.3, 19.0, 22.7, 18.4, 3.5, 6.5, 13.2, 21.4, 20.1, 22.7, 3.0, 5.5, 21.0, 19.4, 22.6, 19.0, 3.6, 36.7, 27.8, 7.6, 3.6, 6.0, 3.3, 40.0, 39.9, 31.0, 8.0, 5.7, 38.9, 37.0, 38.4, 35.3, 38.2, 37.0, 37.1, 36.9, 37.3, 35.1, 36.9, 5.0, 2.2, 5.6, 18.3]
champions_to_churn=[51.6, 19.1, 49.4, 53.8, 48.8, 50.2, 47.9, 48.5, 48.4, 47.4, 47.8, 45.4, 47.4, 46.0, 27.1, 52.4, 49.0, 47.5, 47.8, 48.8, 13.2, 46.2, 48.3, 42.8, 48.5, 46.1, 18.1, 28.6, 17.0, 47.7, 17.5, 46.9, 15.1, 16.6, 19.4, 15.9, 42.8, 16.6, 18.1, 16.9, 18.0, 20.1, 20.2, 21.4, 20.0, 18.9, 19.3, 20.0, 16.9, 51.1, 20.4, 51.0]

churn_to_sleeping= [15.0, 11.3, 11.3, 11.3, 11.5, 11.8, 12.5, 12.1, 11.8, 12.3, 12.4, 13.6, 14.1, 17.3, 13.7, 12.2, 12.2, 11.7, 11.7, 11.8, 12.5, 13.8, 11.9, 15.0, 11.7, 13.1, 10.5, 7.6, 9.0, 9.4, 8.4, 9.6, 9.5, 9.6, 10.2, 8.9, 10.9, 11.5, 11.5, 11.5, 11.5, 11.3, 11.5, 11.4, 10.8, 11.0, 11.5, 11.3, 11.5, 11.6, 10.5, 11.0]
churn_to_loyal=[2.3, 1.0, 1.0, 0.9, 0.9, 1.0, 1.0, 1.1, 0.9, 1.0, 0.9, 1.1, 1.3, 3.8, 1.3, 0.9, 1.0, 0.9, 0.9, 2.0, 1.1, 0.4, 0.9, 1.6, 0.9, 2.1, 2.1, 1.8, 0.8, 2.0, 0.7, 2.2, 1.9, 1.9, 2.1, 2.8, 2.0, 2.2, 2.1, 1.9, 2.0, 2.1, 2.1, 2.1, 2.0, 2.0, 2.1, 2.0, 1.0, 2.2, 0.9, 0.9]
churn_to_champions=[0.7, 1.5, 1.9, 1.6, 1.8, 1.8, 1.9, 1.8, 1.8, 1.9, 1.7, 2.4, 2.5, 0.7, 2.3, 1.7, 1.9, 1.7, 1.9, 0.3, 2.0, 2.1, 1.9, 2.2, 1.8, 0.4, 0.3, 0.2, 1.9, 0.4, 1.7, 0.3, 0.3, 0.4, 0.3, 0.9, 0.3, 0.3, 0.4, 0.4, 0.3, 0.3, 0.4, 0.3, 0.3, 0.3, 0.4, 0.4, 2.1, 0.4, 2.1, 2.1]
churn_to_churn=[82.0, 86.2, 85.9, 86.1, 85.8, 85.4, 84.6, 85.0, 85.5, 84.8, 84.9, 82.9, 82.2, 78.2, 82.7, 85.1, 84.9, 85.7, 85.6, 85.9, 84.4, 83.7, 85.3, 81.1, 85.6, 84.4, 87.1, 90.4, 88.3, 88.2, 89.2, 87.9, 88.3, 88.1, 87.4, 87.4, 86.7, 85.9, 86.0, 86.3, 86.2, 86.3, 86.0, 86.1, 86.9, 86.6, 86.0, 86.3, 85.4, 85.7, 86.5, 86.0]


In [ ]:
flows=pd.DataFrame()

#flows['sleeping_to_sleeping']=sleeping_to_sleeping
flows['sleeping_to_loyal']=sleeping_to_loyal
flows['sleeping_to_champions']=sleeping_to_champions
flows['sleeping_to_churn']=sleeping_to_churn

flows['loyal_to_sleeping']=loyal_to_sleeping
#flows['loyal_to_loyal']=loyal_to_loyal
flows['loyal_to_champions']=loyal_to_champions
flows['loyal_to_churn']=loyal_to_churn

flows['champions_to_sleeping']=champions_to_sleeping
flows['champions_to_loyal']=champions_to_loyal
#flows['champions_to_champions']=champions_to_champions
flows['champions_to_churn']=champions_to_churn

flows['churn_to_sleeping']=churn_to_sleeping
flows['churn_to_loyal']=churn_to_loyal
flows['churn_to_champions']=churn_to_champions
#flows['churn_to_churn']=churn_to_churn

In [ ]:
flows1=flows.iloc[1:,:]

In [ ]:
flows.shape

(52, 16)

In [ ]:
new_df=flows.join(stat_promo)
new_df.drop(columns='week', inplace=True)
new_df

,sleeping_to_loyal,sleeping_to_champions,sleeping_to_churn,loyal_to_sleeping,loyal_to_champions,loyal_to_churn,champions_to_sleeping,champions_to_loyal,champions_to_churn,churn_to_sleeping,churn_to_loyal,churn_to_champions,count_promo_All,count_promo_Billboards,count_promo_Facades,count_promo_Seasonal,count_promo_Biweekly
0,5.0,5.0,50.5,40.3,19.5,34.0,26.0,18.8,51.6,15.0,2.3,0.7,127,7,33,87,0
1,7.7,3.6,55.7,27.0,17.1,50.0,28.2,47.4,19.1,11.3,1.0,1.5,236,10,94,87,45
2,5.8,4.6,56.3,32.8,3.7,25.0,23.4,6.5,49.4,11.3,1.0,1.9,456,14,83,314,45
3,6.0,4.1,57.5,33.1,5.4,23.3,24.4,4.9,53.8,11.3,0.9,1.6,445,46,45,232,122
4,6.4,4.1,56.1,33.0,5.0,22.1,26.9,6.0,48.8,11.5,0.9,1.8,497,38,30,352,77
5,6.2,4.4,56.8,32.3,5.1,25.2,24.9,6.0,50.2,11.8,1.0,1.8,594,55,39,255,245
6,6.7,4.2,56.5,33.1,5.5,25.7,28.6,5.3,47.9,12.5,1.0,1.9,598,41,62,327,168
7,6.5,4.0,56.5,35.2,4.2,23.6,26.0,6.1,48.5,12.1,1.1,1.8,542,42,77,187,236
8,6.7,4.5,56.1,33.2,4.6,24.1,26.3,5.4,48.4,11.8,0.9,1.8,325,6,59,192,68
9,6.7,4.6,54.4,33.3,5.5,22.0,25.5,5.7,47.4,12.3,1.0,1.9,356,10,84,77,185


In [ ]:
new_df.to_csv('flow_не_все.csv')